In [1]:
import os

In [9]:
os.chdir("../")

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source: str
    local_data_dir: Path
    unzip_dir: Path

In [11]:
from projectKidney.constants import *
from projectKidney.utils.common import read_yaml, create_directories

In [15]:
class configurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH, 
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source=config.source_URL,
            local_data_dir=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )
        
        return data_ingestion_config
    
    

In [16]:
import os
import zipfile
import gdown
from projectKidney import logger
from projectKidney.utils.common import get_size


class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self)->str:

        try:
            url = self.config.source
            zip_dir = self.config.local_data_dir
            os.makedirs("artifacts/data_ingestion", exist_ok =True)
            logger.info(f"Downloading data from {url} into {zip_dir}")

            file_id = url.split("/")[-2]
            prefix = ""
            gdown.download(prefix+file_id, zip_dir)

            logger.info(f"Data Downloaded to {zip_dir}")

        except Exception as e:
            raise e  
        
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_dir, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [18]:
try:
    config = configurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-31 15:48:03,731: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-31 15:48:03,736: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-31 15:48:03,739: INFO: common: created directory at: artifacts]
[2025-03-31 15:48:03,742: INFO: common: created directory at: artifacts/data_ingestion]
